In [1]:
pip install codecarbon

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
! codecarbon init


Welcome to CodeCarbon, here is your experiment id:
73c54d65-6fc8-4a34-bef1-c56b3a73de67

CodeCarbon automatically added this id to your local config: ./.codecarbon.config



In [3]:
## CNN 
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = './data/CNN/train_images'
test_dir = './data/CNN/test_images'

transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

In [4]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model_3_layers_no_augmentation = Net().to(device)

In [8]:
def train_NN(n_epochs, optimizer, model, criterion):
    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # print statistics after every epoch
        print(f'[{epoch}, {i + 1:5d}] loss: {running_loss / len(train_loader):.3f}')

    print('Finished Training')

In [10]:
print(len(train_loader))

2270


In [12]:
import torch.optim as optim
from codecarbon import EmissionsTracker

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model_3_layers_no_augmentation.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20

tracker = EmissionsTracker()
tracker.start()
train_NN(n_epochs, optimizer_SGD, model_3_layers_no_augmentation, criterion)
emissions: float = tracker.stop()
print(emissions)



[codecarbon INFO @ 15:06:27] [setup] RAM Tracking...
[codecarbon INFO @ 15:06:27] [setup] GPU Tracking...
[codecarbon INFO @ 15:06:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:06:27] [setup] CPU Tracking...
[codecarbon WARNING @ 15:06:27] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:06:29] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-10900 CPU @ 2.80GHz
[codecarbon INFO @ 15:06:29] >>> Tracker's metadata:
[codecarbon INFO @ 15:06:29]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 15:06:29]   Python version: 3.9.12
[codecarbon INFO @ 15:06:29]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 15:06:29]   Available RAM : 31.742 GB
[codecarbon INFO @ 15:06:29]   CPU count: 20
[codecarbon INFO @ 15:06:29]   CPU model: Intel(R) Core(TM) i9-10900 CPU @ 2.80GHz
[codecarbon INFO @ 15:06:29]   GPU count: 1
[codecarbon INFO @ 15:06:29]   GPU model: 1 x NVIDIA GeForce RTX 3060
[codecarbon INFO @ 15:06:44] Energy con

[1,  2270] loss: 0.595


[codecarbon INFO @ 15:08:15] Energy consumed for RAM : 0.000347 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:08:15] Energy consumed for all GPUs : 0.000300 kWh. Total GPU Power : 9.367924945983011 W
[codecarbon INFO @ 15:08:15] Energy consumed for all CPUs : 0.000948 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:08:15] 0.001596 kWh of electricity used since the beginning.


[2,  2270] loss: 0.237


[codecarbon INFO @ 15:08:30] Energy consumed for RAM : 0.000397 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:08:30] Energy consumed for all GPUs : 0.000376 kWh. Total GPU Power : 18.09472118182884 W
[codecarbon INFO @ 15:08:30] Energy consumed for all CPUs : 0.001084 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:08:30] 0.001857 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 15:13:00] Energy consumed for RAM : 0.001290 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:13:00] Energy consumed for all GPUs : 0.001602 kWh. Total GPU Power : 18.253073302573206 W
[codecarbon INFO @ 15:13:00] Energy consumed for all CPUs : 0.003523 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:13:00] 0.006415 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:13:15] Energy consumed for RAM : 0.001340 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:13:15] Energy consumed for all GPUs : 0.001669 kWh. Total GPU Power : 16.00110370002785 W
[codecarbon INFO @ 15:13:15] Energy consumed for all CPUs : 0.003658 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:13:15] 0.006667 kWh of electricity used since the beginning.


[19,  2270] loss: 0.010


[codecarbon INFO @ 15:13:30] Energy consumed for RAM : 0.001389 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:13:30] Energy consumed for all GPUs : 0.001738 kWh. Total GPU Power : 16.443736349515582 W
[codecarbon INFO @ 15:13:30] Energy consumed for all CPUs : 0.003794 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:13:30] 0.006920 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:13:33] Energy consumed for RAM : 0.001398 kWh. RAM Power : 11.903344631195068 W
[codecarbon INFO @ 15:13:33] Energy consumed for all GPUs : 0.001753 kWh. Total GPU Power : 19.44477847357605 W
[codecarbon INFO @ 15:13:33] Energy consumed for all CPUs : 0.003819 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 15:13:33] 0.006970 kWh of electricity used since the beginning.


[20,  2270] loss: 0.009
Finished Training
0.0004697101059861965


RuntimeError: Parent directory models does not exist.